In [1]:
import os
from utils import *
import optuna
from Recommenders.Recommender_import_list import *
import scipy.sparse as sps
from tqdm import tqdm

Tensorflow is not available


c:\Users\francesco\miniconda3\envs\RecSysFramework\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
data, usermap, itemmap, users = load_data2()
data_train, data_val = split_data2(data, 0.2)
data_train_val = data_train + data_val

In [ ]:
# 0.08254950935282762
slimel_study = optuna.load_study(
    study_name="SLIMElastic",
    storage=get_database_url(),
)

# 0.07018143152516958
slimbpr_study = optuna.load_study(
    study_name="SLIMBPR",
    storage=get_database_url(),
)

# 0.08248515814658722
p3alpha_study = optuna.load_study(
    study_name="P3Alpha",
    storage=get_database_url(),
)

# 0.082429130748692
easer_study = optuna.load_study(
    study_name="Easer",
    storage=get_database_url(),
)

# 0.06470798788105654
userknncf_study = optuna.load_study(
    study_name="UserKNNCF",
    storage=get_database_url(),
)

# 0.07684279728158312
itemknncf_study = optuna.load_study(
    study_name="ItemKNNCF",
    storage=get_database_url(),
)

# 0.08511222934213017
p3beta_study = optuna.load_study(
    study_name="P3Beta",
    storage=get_database_url(),
)

# 0.08513855277446945
hybrid_itemknncf_p3beta_study = optuna.load_study(
    study_name="ItemKNNCF + P3Beta",
    storage=get_database_url(),
)

# 0.08660457653642996
hybrid_itemknncf_userknncf_p3beta_study = optuna.load_study(
    study_name="ScoresHybrid",
    storage=get_database_url(),
)

# 0.08268787988452277
hybrid_slimel_slimbpr_study = optuna.load_study(
    study_name="SLIMElastic+SLIMBPR",
    storage=get_database_url(),
)

# 0.085366019104557
hybrid_p3beta_easer_study = optuna.load_study(
    study_name="P3Beta+Easer",
    storage=get_database_url(),
)

# 0.08666846078257197
hybrid_slimel_p3beta_study = optuna.load_study(
    study_name="SLIMElastic+P3Beta",
    storage=get_database_url(),
)

# 0.08350570630637119
hybrid_slimel_p3beta_easer_study = optuna.load_study(
    study_name="SLIMElastic+P3Beta+Easer",
    storage=get_database_url(),
)

# 0.08368146099525228
hybrid_slimel_p3beta_itemknncf_study = optuna.load_study(
    study_name="SLIMElastic+P3Beta+ItemKNNCF",
    storage=get_database_url(),
)

# 0.08606971380880395
hybrid_easer_slimel_itemknncf_slimbpr_study = optuna.load_study(
    study_name="Easer+SlimElastic+ItemKNNCF+SLIMBPR",
    storage=get_database_url(),
)

# 0.08555814709467932
hybrid_slimbpr_p3beta_easer_study = optuna.load_study(
    study_name="P3Beta+Easer+SLIMBPR",
    storage=get_database_url(),
)

# 0.08339769978001665
hybrid_easer_slimel_study = optuna.load_study(
    study_name="Easer+SlimElastic",
    storage=get_database_url(),
)

In [ ]:
slimel_recommender = MultiThreadSLIM_SLIMElasticNetRecommender(data_train, verbose=False)
slimel_recommender.fit(**slimel_study.best_params)

# [alpha: 0.0023866170167418325, topK: 396, l1_ratio: 2.5949057850541665e-07]

In [ ]:
slimbpr_recommender = SLIM_BPR_Cython(data_train, verbose=False)
slimbpr_recommender.fit(**slimbpr_study.best_params)

# [epochs: 490, symmetric: False, lambda_i: 0.04973485595359249, lambda_j: 1.8772031164012726e-07, learning_rate: 0.008459713940705181, topK: 23, sgd_mode: adagrad, gamma: 0.0036083269768859244, beta_1: 0.010947253104592736, beta_2: 1.7512021923757238e-05]

In [ ]:
p3alpha_recommender = P3alphaRecommender(data_train, verbose=False)
p3alpha_recommender.fit(**p3alpha_study.best_params)

# [topK: 61, alpha: 0.4121129634953655, implicit: True, normalize_similarity: True]

In [ ]:
easer_recommender = EASE_R_Recommender(data_train, verbose=False)
easer_recommender.fit(**easer_study.best_params)

# [topK: 72, l2_norm: 36.47305040353163]

In [ ]:
userknncf_recommender = UserKNNCFRecommender(data_train, verbose=False)
userknncf_recommender.fit(**userknncf_study.best_params)

# [topK: 281, similarity: cosine, normalize: True, feature_weighting: TF-IDF]

In [ ]:
itemknncf_recommender = ItemKNNCFRecommender(data_train, verbose=False)
itemknncf_recommender.fit(**itemknncf_study.best_params)

# [topK: 180, similarity: cosine, normalize: True, feature_weighting: TF-IDF]

In [ ]:
p3beta_recommender = RP3betaRecommender(data_train, verbose=False)
p3beta_recommender.fit(**p3beta_study.best_params)

# [topK: 35, alpha: 0.3598110596257423, beta: 0.1558338653772352, implicit: True, normalize_similarity: True]

In [ ]:
hybrid_itemknncf_p3beta_recommender = DifferentLossScoresHybridRecommender(
    data_train,
    p3beta_recommender,
    itemknncf_recommender,
)
hybrid_itemknncf_p3beta_recommender.fit(**hybrid_itemknncf_p3beta_study.best_params)

# [alpha: 0.061089174221146764]

In [ ]:
recommenders = [
    {
        "recommender": ItemKNNCFRecommender,
        "params": itemknncf_study.best_params,
    },
    {
        "recommender": UserKNNCFRecommender,
        "params": userknncf_study.best_params,
    },
    {
        "recommender": RP3betaRecommender,
        "params": p3beta_study.best_params,
    }
]

hybrid_itemknncf_userknncf_p3beta_recommender = ScoresHybridRecommender(data_train, recommenders)
hybrid_itemknncf_userknncf_p3beta_recommender.prefit()
hybrid_itemknncf_userknncf_p3beta_recommender.fit(**hybrid_itemknncf_userknncf_p3beta_study.best_params)

# [weight_0: 0.0007057494959371759, weight_1: 0.0010435587541626084, weight_2: 0.5557013992211137]

In [ ]:
hybrid_slimel_slimbpr_recommender = DifferentLossScoresHybridRecommender(
    data_train,
    slimel_recommender,
    slimbpr_recommender,
)
hybrid_slimel_slimbpr_recommender.fit(**hybrid_slimel_slimbpr_study.best_params)

# [norm: 1, alpha: 0.9711764934578814]

In [ ]:
hybrid_p3beta_easer_recommender = DifferentLossScoresHybridRecommender(
    data_train,
    p3beta_recommender,
    easer_recommender,
)
hybrid_p3beta_easer_recommender.fit(**hybrid_p3beta_easer_study.best_params)

# [norm: None, alpha: 0.5641427244696609]

In [ ]:
hybrid_slimel_p3beta_recommender = DifferentLossScoresHybridRecommender(
    data_train,
    slimel_recommender,
    p3beta_recommender,
)
hybrid_slimel_p3beta_recommender.fit(**hybrid_slimel_p3beta_study.best_params)

# [norm: 2, alpha: 0.6182214281226774]

In [ ]:
hybrid_slimel_p3beta_easer_recommender = DifferentLossScoresHybridRecommender(
    data_train,
    hybrid_slimel_p3beta_recommender,
    easer_recommender,
)
hybrid_slimel_p3beta_easer_recommender.fit(**hybrid_slimel_p3beta_easer_study.best_params)

# [norm: None, alpha: 0.8485943767022647]

In [ ]:
hybrid_slimel_p3beta_itemknncf_recommender = DifferentLossScoresHybridRecommender(
    data_train,
    hybrid_slimel_p3beta_recommender,
    itemknncf_recommender,
)
hybrid_slimel_p3beta_itemknncf_recommender.fit(**hybrid_slimel_p3beta_itemknncf_study.best_params)

# [weight_0: 0.0007057494959371759, weight_1: 0.0010435587541626084, weight_2: 0.5557013992211137]

In [ ]:
hybrid_easer_slimel_itemknncf_slimbpr_recommender_1 = DifferentLossScoresHybridRecommender(data_train, easer_recommender, slimel_recommender)
hybrid_easer_slimel_itemknncf_slimbpr_recommender_1.fit(norm=hybrid_easer_slimel_itemknncf_slimbpr_study.best_params['norm'], alpha=hybrid_easer_slimel_itemknncf_slimbpr_study.best_params['alpha'])
hybrid_easer_slimel_itemknncf_slimbpr_recommender_2 = DifferentLossScoresHybridRecommender(data_train, hybrid_easer_slimel_itemknncf_slimbpr_recommender_1, itemknncf_recommender)
hybrid_easer_slimel_itemknncf_slimbpr_recommender_2.fit(norm=hybrid_easer_slimel_itemknncf_slimbpr_study.best_params['norm'], alpha=hybrid_easer_slimel_itemknncf_slimbpr_study.best_params['beta'])
hybrid_easer_slimel_itemknncf_slimbpr_recommender = DifferentLossScoresHybridRecommender(data_train, hybrid_easer_slimel_itemknncf_slimbpr_recommender_2, slimbpr_recommender)
hybrid_easer_slimel_itemknncf_slimbpr_recommender.fit(norm=hybrid_easer_slimel_itemknncf_slimbpr_study.best_params['norm'], alpha=hybrid_easer_slimel_itemknncf_slimbpr_study.best_params['gamma'])

# [norm: None, alpha: 0.07677940193059153, beta: 0.6475804621808522, gamma: 0.9786981109318013]

In [ ]:
hybrid_slimbpr_p3beta_easer_recommender = DifferentLossScoresHybridRecommender(
    data_train,
    slimbpr_recommender,
    hybrid_p3beta_easer_recommender,
)
hybrid_slimbpr_p3beta_easer_recommender.fit(
    hybrid_slimbpr_p3beta_easer_study.best_params['norm'],
    hybrid_slimbpr_p3beta_easer_study.best_params['alpha'],
)

# [norm: -inf, alpha: 0.9415797004214617, beta: 0.26986192573566387]

In [ ]:
hybrid_easer_slimel_recommender = DifferentLossScoresHybridRecommender(
    data_train,
    easer_recommender,
    slimel_recommender,
)
hybrid_easer_slimel_recommender.fit(**hybrid_easer_slimel_study.best_params)

# [norm: inf, alpha: 0.08364695105189879]

In [ ]:
n_users, n_items = data_train_val.shape

training_dataframe = pd.DataFrame(index=range(0,n_users), columns = ["ItemID"])
training_dataframe.index.name='UserID'

In [ ]:
cutoff = 50

for user_id in range(n_users):    
    recommendations = hybrid_slimel_p3beta_recommender.recommend(user_id, cutoff = cutoff, remove_seen_flag=True)
    training_dataframe.loc[user_id, "ItemID"] = recommendations

In [ ]:
training_dataframe = training_dataframe.explode("ItemID")

In [ ]:
data_val_coo = sps.coo_matrix(data_val)
correct_recommendations = pd.DataFrame({"UserID": data_val_coo.row,
                                        "ItemID": data_val_coo.col})

In [ ]:
training_dataframe = pd.merge(training_dataframe, correct_recommendations, on=['UserID','ItemID'], how='left', indicator='Exist')

In [ ]:
training_dataframe["Label"] = training_dataframe["Exist"] == "both"
training_dataframe.drop(columns = ['Exist'], inplace=True)

In [ ]:
topPop = TopPop(data_train)
topPop.fit()

other_algorithms = {
    "TopPop": topPop,
    "slimel": slimel_recommender,
    "slimbpr": slimbpr_recommender,
    "p3alpha": p3alpha_recommender,
    "easer": easer_recommender,
    "userknncf": userknncf_recommender,
    "itemknncf": itemknncf_recommender,
    "p3beta": p3beta_recommender,
    "hybrid_itemknncf_p3beta": hybrid_itemknncf_p3beta_recommender,
    "hybrid_itemknncf_userknncf_p3beta": hybrid_itemknncf_userknncf_p3beta_recommender,
    "hybrid_slimel_slimbpr": hybrid_slimel_slimbpr_recommender,
    "hybrid_p3beta_easer": hybrid_p3beta_easer_recommender,
    "hybrid_slimel_p3beta_easer": hybrid_slimel_p3beta_easer_recommender,
    "hybrid_slimel_p3beta_itemknncf": hybrid_slimel_p3beta_itemknncf_recommender,
    "hybrid_easer_slimel_itemknncf_slimbpr": hybrid_easer_slimel_itemknncf_slimbpr_recommender,
    "hybrid_slimbpr_p3beta_easer": hybrid_slimbpr_p3beta_easer_recommender,
    "hybrid_easer_slimel": hybrid_easer_slimel_recommender,
}

In [ ]:
training_dataframe = training_dataframe.set_index('UserID')

for user_id in tqdm(range(n_users)):  
    for rec_label, rec_instance in other_algorithms.items():
        
        item_list = training_dataframe.loc[user_id, "ItemID"].values.tolist()
        
        all_item_scores = rec_instance._compute_item_score([user_id], items_to_compute = item_list)

        training_dataframe.loc[user_id, rec_label] = all_item_scores[0, item_list] 

training_dataframe = training_dataframe.reset_index()
training_dataframe = training_dataframe.rename(columns = {"index": "UserID"})

In [ ]:
item_popularity = np.ediff1d(sps.csc_matrix(data_train).indptr)

training_dataframe['item_popularity'] = item_popularity[training_dataframe["ItemID"].values.astype(int)]

In [ ]:
user_popularity = np.ediff1d(sps.csr_matrix(data_train).indptr)

training_dataframe['user_profile_len'] = user_popularity[training_dataframe["UserID"].values.astype(int)]

In [ ]:
training_dataframe = training_dataframe.sort_values("UserID").reset_index()
training_dataframe.drop(columns = ['index'], inplace=True)

In [ ]:
groups = training_dataframe.groupby("UserID").size().values

In [ ]:
study = optuna.create_study(
    study_name="XGBoostWin",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

In [ ]:
def evaluator(data_train, train_recs, XGB_model, training_dataframe, data_test: sp.csr_matrix, recommendation_length: int = 10):
    accum_precision = 0
    accum_recall = 0
    accum_map = 0

    num_users = data_train.shape[0]

    num_users_evaluated = 0
    num_users_skipped = 0
    for user_id in range(num_users):
        user_profile_start = data_test.indptr[user_id]
        user_profile_end = data_test.indptr[user_id+1]

        relevant_items = data_test.indices[user_profile_start:user_profile_end]

        if relevant_items.size == 0:
            num_users_skipped += 1
            continue
        
        predict = train_recs[train_recs["UserID"] == user_id]
        predictions = XGB_model.predict(predict)
        user_ids = predict["UserID"].values
        item_ids = predict["ItemID"].values
        predictions = pd.DataFrame({"UserID": user_ids, "ItemID": item_ids, "Predictions": predictions})
        predictions = predictions.sort_values(by="Predictions", ascending=False)
        recommendations = np.array(predictions["ItemID"].values[:recommendation_length])

        accum_precision += precision(recommendations, relevant_items)
        accum_recall += recall(recommendations, relevant_items)
        accum_map += mean_average_precision(recommendations, relevant_items)

        num_users_evaluated += 1


    accum_precision /= max(num_users_evaluated, 1)
    accum_recall /= max(num_users_evaluated, 1)
    accum_map /=  max(num_users_evaluated, 1)

    return accum_precision, accum_recall, accum_map, num_users_evaluated, num_users_skipped


In [ ]:
train_recs = training_dataframe.drop(columns=["Label"])
train_labs = training_dataframe["Label"]

In [ ]:
from xgboost import XGBRanker

def objective(trial):
    objective = trial.suggest_categorical("objective", ["rank:ndcg", "rank:map", "rank:pairwise"])
    n_estimators = trial.suggest_int("n_estimators", 5, 500)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
    reg_alpha = trial.suggest_float("reg_alpha", 1e-4, 1e-1, log=True)
    reg_lambda = trial.suggest_float("reg_lambda", 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int("max_depth", 1, 10)
    max_leaves = trial.suggest_int("max_leaves", 1, 10)
    grow_policy = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    booster = trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"])

    XGB_model = XGBRanker(
        tree_method="hist",
        objective=objective,
        n_estimators=int(n_estimators),
        random_state=None,
        learning_rate=learning_rate,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        max_depth=int(max_depth),
        max_leaves=int(max_leaves),
        grow_policy=grow_policy,
        booster=booster,
        verbosity=0, # 2 if self.verbose else 0,
    )
    XGB_model.fit(
        train_recs,
        train_labs,
        group=groups,
        verbose=False
    )
    _, _, ev_map, _, _ = evaluator(data_train, train_recs, XGB_model, training_dataframe, data_val)
    
    return ev_map

study.optimize(objective, n_trials=1000)

In [ ]:
from xgboost import XGBRanker

XGB_model = XGBRanker(
    **study.best_params,
    verbosity=0, # 2 if self.verbose else 0,
    enable_categorical=True,
)
XGB_model.fit(
    train_recs,
    train_labs,
    group=groups,
    verbose=False
)

In [ ]:
def submission2(train_recs, XGB_model, users, usermap, itemmap, data_train_val):
    
    toppoprecommender = TopPop(data_train_val)
    toppoprecommender.fit()
    
    with open('data/results.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['user_id', 'item_list'])
        for user_id in users:
            predict = train_recs[train_recs["UserID"] == user_id]
            user_ids = predict["UserID"].values
            item_ids = predict["ItemID"].values
            predictions = XGB_model.predict(predict)
            if not list(predictions):
                predictions = toppoprecommender.recommend(0, cutoff = 10)
                item_list = [itemmap[i] for i in predictions]
                writer.writerow([usermap[user_id], ' '.join(map(str, item_list))])
                continue
            predictions = pd.DataFrame({"UserID": user_ids, "ItemID": item_ids, "Predictions": predictions.ravel()})
            predictions = predictions.sort_values(by="Predictions", ascending=False)
            for i in range(10):
                if predictions["ItemID"].values[i] in data_train_val[user_id].indices:
                    predictions["Predictions"].values[i] = -10
            recommendations = np.array(predictions["ItemID"].values[:10])
            item_list = [itemmap[i] for i in recommendations]
            writer.writerow([usermap[user_id], ' '.join(map(str, item_list))])
                

In [ ]:
submission2(train_recs, XGB_model, users, usermap, itemmap, data_train_val)